# Methanotrophy

In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
from utils import * 
from matrices import *
from ordination import *
from plotting import *
import os

## Loading data

In [3]:
DATA_DIR = '/home/prichter/Documents/data/methanotrophy'

In [4]:
metadata_df = dataframe_from_metadata(os.path.join(DATA_DIR, 'metadata.csv'))
counts_df = dataframe_from_counts(os.path.join(DATA_DIR, 'counts.tsv'))
taxonomy_df = dataframe_from_taxonomy(os.path.join(DATA_DIR, 'taxonomy.tsv'))

print('Fields in metadata.csv:', ', '.join(metadata_df.columns))
print('Fields in counts.tsv:', ', '.join(counts_df.columns))
print('Fields in taxonomy.tsv:', ', '.join(taxonomy_df.columns))

Fields in metadata.csv: sample, serial_code, soil_depth, site, season, flux_ch4, temp_air, temp_soil, water_content, bulk_density
Fields in counts.tsv: asv, sample, count
Fields in taxonomy.tsv: domain, phylum, class, order, family, genus, species, domain_sub, phylum_sub, class_sub, order_sub, family_sub, genus_sub, species_sub, asv


In [5]:
missing_samples = set(counts_df['sample'].values) - set(metadata_df['sample'].values)
print(f"Samples {', '.join(list(missing_samples))} are present in the counts.tsv file, but not in metadata.")
print(f"Dropping {counts_df['sample'].isin(missing_samples).values.sum()} rows in the counts data which do not have associated metadata." )
counts_df = counts_df[~counts_df['sample'].isin(missing_samples)] # Remove rows in counts_df which don't have associated metadata.

# Combine the data across files into a single DataFrame. Drop the samples column, which is redundant with the serial code. 
df = counts_df.merge(metadata_df, on=['sample']).merge(taxonomy_df, on=['asv'], how='left').drop(columns='sample')

# Save the cleaned-up data to a CSV file. 
df.to_csv(f'{DATA_DIR}/data.csv', index=False)

Samples HDK22-KML-sand-dry, HDK22-KML-sand-wet are present in the counts.tsv file, but not in metadata.
Dropping 192136 rows in the counts data which do not have associated metadata.


In [6]:
def create_asv_matrix(df:pd.DataFrame) -> pd.DataFrame:
    '''Convert the DataFrame containing the sample metadata and ASV counts into an ASV table (with columns
    as ASVs, and rows as samples). Each cell contains the raw count for that particular ASV in the sample.''' 
    # Accumulate all of the metadata into a separate DataFrame. 
    metadata = df[[col for col in df.columns if col not in ['count']]]

    df = df[['serial_code', 'asv', 'count']]
    df = df.groupby(by=['serial_code', 'asv']).sum()
    df = df.reset_index() # Converts the multi-level index to categorical columns. 
    df = df.pivot(columns=['asv'], index=['serial_code'], values=['count'])
    # Reset column labels, which were weird because of the multi-indexing. 
    df.columns = df.columns.get_level_values('asv').values

    return AsvMatrix(df, metadata=metadata)

## Analysis

### Correspondence Analysis 

In [7]:
# Generating the count matrix is currently extremely slow. Possibly a way to speed it up?
m = create_asv_matrix(pd.read_csv(f'{DATA_DIR}/data.csv', nrows=1000000))
m.filter_read_depth(5000)
m.row_labels

matrices.AsvMatrix.filter_read_depth: Discarding 1 samples with read depth less than 5000.


array([65, 66, 67, 68, 69])

In [8]:
tm = m.get_taxonomy_matrix()

In [ ]:
ca = CorrespondenceAnalysis()

### Which environmental factors are predictive of methane flux?

In [10]:
# plot_rarefaction_curves(m, path='/home/prichter/Documents/methanotrophy/figures/rarefaction_curves.png')